<a href="https://colab.research.google.com/github/danasaur/nlp/blob/main/notebooks/poetry_setfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--<badge>--><a href="https://colab.research.google.com/github/huggingface/workshops/blob/main/fewshot-learning-in-production/setfit-optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

In [ ]:
human_haikus = [
    "The feeling of plush, The perfect repose of sole, Comes with the meow",
    "Sun lit finches splash, in a rain shower puddle, cats wait in shadow.", 
    "A cat’s just a cat, til he has master at hand, then he is the boss!",
    "Cats are cute and sweet, Loving and Playful, like me! But, Dogs are better!",
    "Grey periscope tail, Silently signals presence, The cat has come home.",
    "Lazy cat sleeping, In front of the warm fire, As the embers cool.",
    "Green eyed yellow cat, Stalks through tall prairie grasses, Cat-like bounds and leaps",
    "Handsome, nonchalant, Seems to care less about me, Then he comes purring"]

chatgpt_haikus = [
"Graceful feline form, Silent hunter in the night, Purring ball of love",
"Eyes gleaming with light, Paws padding on hardwood floors, Soft purrs fill the air",
"Mysterious cat, Lurking in the shadows still, Whiskers twitch with glee",
"Playful kitten leaps, Chasing after bits of string, Joyful laughter rings",
"Midnight wanderer, Prowling through the garden maze, Moonlit fur aglow",
"Purring in my lap, Contented cat dozes off, Warmth shared on this chair",
"Curled up by the fire, Softly snoozing through the night, Dreaming of mice caught",
"Whiskers, tail, and paws, Furry friend, always by my side, Loyal to the end"
]

## 1. Setup development environment

Our first step is to install SetFit. Running the following cell will install all the required packages for us, including Intel Neural Compressor and 🤗 Optimum:

In [ ]:
%pip install setfit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB

## 3. Train teacher and student baselines

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files=["chatgpt_or_human_haikus.csv"])
dataset

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 16
    })
})

In [ ]:
from datasets import load_dataset

test_dataset = load_dataset("csv", data_files=["test_haikus.csv"])
test_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e1e093e8ffd914c1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'],
        num_rows: 1
    })
})

In [ ]:
# test_dataset = test_dataset["train"]

KeyError: ignored

In [ ]:
type(dataset)

datasets.dataset_dict.DatasetDict

To simulate working in a few-shot regime, we'll randomly sample from the training set. We'll also need 1,000 unlabeled examples for the knowledge distillation step, so let's split the training data into separate splits to avoid any data leakage:

In [ ]:
train_dataset_teacher = dataset["train"]

In [ ]:
# from setfit import sample_dataset

# # Create 2 splits: one for few-shot training, the other for knowledge distillation
# train_dataset = dataset["train"].train_test_split(seed=42)
# # Sample 8 examples / class for fine-tuning
# train_dataset_teacher = sample_dataset(train_dataset["train"])
# # Select 1000 unlabeled examples for knowledge distillation
# train_dataset_student = train_dataset["test"].select(range(1000))
# # Define the test set for evaluation
# test_dataset = dataset["test"]

In [ ]:
type(train_dataset_teacher)

datasets.arrow_dataset.Dataset

In [ ]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 30000
    })
})

In [ ]:
train_dataset_teacher.features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'label_text': Value(dtype='string', id=None)}

In [ ]:
train_dataset_teacher.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)}

Now that we've got out training and test sets, let's train some SetFit models! With the `SetFitModel` and `SetFitTrainer` classes, this is quite simple:

In [ ]:
from setfit import SetFitModel, SetFitTrainer

# Load pretrained model from the Hub
teacher_model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2"
)
# Create trainer
teacher_trainer = SetFitTrainer(
    model=teacher_model, train_dataset=train_dataset_teacher
)
# Train!
teacher_trainer.train()

# # Evaluate!
# pb = PerformanceBenchmark(
#     model=teacher_trainer.model, dataset=test_dataset, optim_type="MPNet (teacher)"
# )
# perf_metrics = pb.run_benchmark()

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num examples = 640
  Num epochs = 1
  Total optimization steps = 40
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
teacher_trainer.model.predict_proba(test_dataset["train"]["text"])

tensor([[0.2136, 0.7864]], dtype=torch.float64)